In [ ]:
import json
import sklearn
import numpy as np
import pandas as pd

from tqdm import tqdm

from itertools import product
from collections.abc import Iterable

from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit
)
from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
    f1_score
)
from sklearn.cross_decomposition import PLSRegression

from imblearn.over_sampling import SMOTE

### **import json**
  json은 JSON 데이터를 읽고 쓸 수 있게 해주는 표준 라이브러리입니다.

### **import sklearn**
  sklearn(scikit learn)은 파이썬 머신러닝 라이브러리로 다양한 머신러닝 알고리즘을 제공합니다.

### **import numpy as np**
  배열을 처리하는 라이브러리로 다차원 배열의 빠른 처리와 배열 broadcasting 기능, 선형 변환 기능을 사용할 수 있습니다. numpy 라이브러리 기능을 사용할 때, 편의성을 위해 np로 호출할 수 있도록 as np를 붙여줍니다.

### **import pandas as pd**
  Pandas는 Series와 DataFrame 자료형을 사용하여 데이터를 처리하는 파이썬 데이터 분석 라이브러리입니다. pandas 함수 호출을 간편하게 처리하기 위해 as pd를 사용합니다.

### **from tqdm import tqdm**
  tqdm은 파이썬의 반복문의 진행 상황을 시각적으로 보여주는 라이브러리입니다.

### **from itertools import product**  
  itertools.product를 사용하여 반복 가능한 변수들의 곱을 계산할 수 있도록 해줍니다. 즉, 반복 가능한 변수들 간의 모든 조합을 생상하는 데 사용할 수 있습니다.

### **from collections.abc import Iterable**
  Iterable은 객체가 반복 가능한지를 확인할 수 있도록 해주는 클래스입니다.

### **from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit**
  StratifiedKFold는 데이터를 계층화된 방법으로 분할하여 교차 검증을 수행할 수 있도록 해주는 클래스입니다.

  StratifiedShuffleSplit는 불균형 분포를 위한 교차 검증을 위해서 계층화된 방식으로 무작위 분할을 통해 Train/Test Set을 생성하는 클래스입니다.

### **from sklearn.metrics import precision_score, recall_score,   accuracy_score, roc_auc_score, f1_score**

  **Precision, Recall, Accurcay, ROC_AUC** 점수모델의 성능을 평가하기 위한 다양한 메트릭을 계산하는 함수들을 import합니다.

### **from sklearn.cross_decomposition import PLSRegression**

  부분 최소 제곱 회귀함수를 수행하는 클래스입니다. 고차원 데이터를 사용하여 선형 회귀를 진행하기 위한 클래스입니다.

### **from imblearn.over_sampling import SMOTE**

  소수 클래스 집합의 데이터를 오버샘플링하기 위한 SMOTE 기법을 사용하는 클래스입니다. 불균형 데이터셋을 처리하기 위해서 소수 클래스의 샘플을 합성하여 추가하는 SMOTE 방식을 사용할 수 있습니다.



In [ ]:
def data_split(X, y, seed):
    sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = seed)

    for train_idx, test_idx in sss.split(X, y):
        train_x = X.iloc[train_idx].reset_index(drop = True)
        train_y = y.iloc[train_idx].reset_index(drop = True)
        test_x = X.iloc[test_idx].reset_index(drop = True)
        test_y = y.iloc[test_idx].reset_index(drop = True)

    return train_x, test_x, train_y, test_y

**data_split** 함수는 데이터를 훈련 세트와 테스트 세트로 분할하기 위한 함수입니다. 이 함수는 **데이터셋 X(특성들), y(타겟 변수),** 그리고 **시드(seed)** 값을 입력으로 받습니다.

**StratifiedShuffleSplit**: 이 클래스는 데이터를 계층화하여 무작위로 섞은 후 훈련 세트와 테스트 세트로 분할합니다.

 아래는 StratifiedShuffleSplit 함수의 입력 파라미터에 대한 설명입니다.
 ```python
 sklearn.model_selection.StratifiedShuffleSplit(n_splits=10, *, test_size=None, train_size=None, random_state=None)

  n_splitsint, default=10
   - 반복과 재구성, 분할 횟수

  test_size floatorint, default= none
   - 테스트 분할에 포함할 데이터 집합의 비율을 나타냅니다.
     Integer 타입의 경우, 세트스 샘플의 절대 수를 나타내고, Float 타입의 경우는 테스트 분할 비율을 의미합니다.

  random_stateint, RandomState instance or None, default=None
   - 훈련과 테스트 셋의 무작위성을 제어합니다. seed를 고정하여 무작위 분할이 아닌 재현 가능한 결과를 만들어 낼 수 있도록 해줍니다.

 # 더 자세한 내용은 (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)을 참조하세요.
 ```
 **data_split** 함수에서 호출한 StratifiedShuffleSplit의 설정은 아래와 같습니다.

  n_splits=1은 데이터 분할을 한 번만 수행함을 의미하고, test_size=0.2는 전체 데이터의 20%를 테스트 세트로 사용하겠다는 것을 나타냅니다. random_state는 seed값을 제공했으므로 분할 시 사용되는 무작위성을 제어합니다.

함수는 분할된 훈련 인덱스(train_idx)와 테스트 인덱스(test_idx)를 사용하여 원본 데이터셋에서 훈련 및 테스트 세트를 추출합니다. DataFrame을 다루는 Pandas 라이브러리의 iloc 메소드를 사용하여 해당 인덱스에 해당하는 데이터를 선택하고, reset_index(drop=True)를 사용하여 인덱스를 초기화합니다. 이를 통해 분할된 데이터셋의 인덱스를 원본 데이터셋의 인덱스와 유지할 수 있습니다.

분할된 훈련 데이터(train_x, train_y)와 테스트 데이터(test_x, test_y)를 반환합니다.

In [ ]:
def parameter_grid(param_dict):
    if not isinstance(param_dict, dict):
        raise TypeError('Parameter grid is not a dict ({!r})'.format(param_dict))

    if isinstance(param_dict, dict):
        for key in param_dict:
            if not isinstance(param_dict[key], Iterable):
                raise TypeError('Parameter grid value is not iterable '
                                '(key={!r}, value={!r})'.format(key, param_dict[key]))

    items = sorted(param_dict.items())
    keys, values = zip(*items)

    params_grid = []
    for v in product(*values):
        params_grid.append(dict(zip(keys, v)))

    return params_grid

**parameter_grid** 함수는 Python Dictionary 형색의 입력 매개변수(**param_dict**)를 받아서 가능한 모든 매개변수 조합을 생성합니다. 위 함수는 머신러닝 모델의 Hyperparameter 최적화 과정에서 사용할 수 있습니다.

```python
  isinstance(object, type)
  
  ininstance() 함수는 입력받은 object가 입력받은 type 유형이라면 True, 그렇지 않다면 False를 반환하는 함수입니다.
```

 위 isinstance함수를 사용하여 사전 형식인지를 검사하고. 만약 사전 타입이 아니라면 TypeError를 발생시킵니다.

 만약 입력받은 param_dict가 사전 형식이라면, 사전의 각 값이 반복 가능한 객체(Iterable)인지 검사합니다. 반복 불가능한 값이 있다면 역시 TypeError를 발생시킵니다.

 이후, 입력받은 param_dict 사전의 키와 값들을 정렬하여 튜플로 분리합니다.

 아래는 product()함수에 대한 설명입니다.
 ```python
  itertools.product(*iterables, repeat=1)

  입력 받은 반복 가능한 변수들의 데카르트 곱을 도출하는 함수입니다,

  위 함수는 중첩된 for-루프와 동등합니다. 예를 들어, product(A, B)는 ((x,y) for x in A for y in B)와 같은 것을 반환합니다.
  중첩된 루프는 매 이터레이션마다 가장 오른쪽 요소가 진행되는 주행 거리계처럼 순환합니다. 이 패턴은 사전식 순서를 만들어서 입력의 이터러블들이 정렬되어 있다면, 곱(product) 튜플이 정렬된 순서로 방출됩니다.

  # 더 자세한 내용은 (https://docs.python.org/ko/3/library/itertools.html)을 참조하세요
 ```
 product(*values)를 사용하여 각 키에 대한 가능한 모든 값의 조합을 생성하고, 이 조합들을 사전 형식으로 다시 매핑하여 결과 리스트(params_grid)에 추가합니다.

결국 이 함수의 최종 결과는 가능한 모든 매개변수 조합을 포함하는 사전의 리스트입니다.

In [ ]:
def multiclass_cross_validation(model, x, y, seed):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

    metrics = ['precision', 'recall', 'f1', 'accuracy']

    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))

    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []

    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []

    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            onehot_train_y = pd.get_dummies(train_y)

            model.fit(train_x, onehot_train_y)

            train_pred = np.argmax(model.predict(train_x), axis = 1)
            val_pred = np.argmax(model.predict(val_x), axis = 1)

        else:
            model.fit(train_x, train_y)

            train_pred = model.predict(train_x)
            val_pred = model.predict(val_x)

        train_precision.append(precision_score(train_y, train_pred, average = 'macro'))
        train_recall.append(recall_score(train_y, train_pred, average = 'macro'))
        train_f1.append(f1_score(train_y, train_pred, average = 'macro'))
        train_accuracy.append(accuracy_score(train_y, train_pred))

        val_precision.append(precision_score(val_y, val_pred, average = 'macro'))
        val_recall.append(recall_score(val_y, val_pred, average = 'macro'))
        val_f1.append(f1_score(val_y, val_pred, average = 'macro'))
        val_accuracy.append(accuracy_score(val_y, val_pred))

    result = dict(zip(train_metrics + val_metrics,
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy),
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy)]))

    return(result)

**multiclass_cross_validation** 함수는 다중 클래스 분류 문제에 대한 교차 검증을 수행하는 함수입니다.이 함수는 **모델(model), 특성 데이터(x), 타겟 데이터(y),** 그리고 **시드 값(seed)**을 입력으로 받습니다.

아래는 **StratifiedKFold**에 대한 설명입니다.

```python
 sklearn.model_selection.StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)

 위 StratifiedKFold는 Train/Test 분할을 위한 indices를 제공합니다.
 이 Cross Validation은 KFold 방식의 변형으로 각 클래스에 대한 비율을 보존하여 KFold를 진행하게 됩니다.

 아래는 입력 Parameter에 대한 설명입니다.

  n_splitsint, default=5
  - Folds의 숫자를 정하는 파라미터로 최소한 2개 이상의 정수를 가져야 한다.

  shufflebool, default=False
  - split 이전에 각 클래스의 샘플들을 섞을지 결정하는 인자입니다. 다만, 분할 내의 샘플들을 섞이지 않습니다.

  random_stateint, RandomState instance or None, default=None
   - shuffle이 참아라면, random_state의 값에 따라서 랜덤 인덱스의 순서에 영향을 미치며, Random_state의 int 값에 따라서 값이 고정되어 재현이 가능해집니다. shuffle을 사용하지 않는다면, None 값을 사용합니다.

 # 더 자세한 내용은 (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)를 참고하세요.
```

이 함수는 **StratifiedKFold**를 사용하여 데이터를 5개의 폴드로 나누고, 각 Fold에서 모델을 훈련 및 검증합니다. 이후, 다양한 평가 지표(정밀도, 재현율, F1 점수, 정확도)의 평균값을 계산하여 훈련 및 검증 성능을 평가합니다.

특히, PLSRegression 모델(부분 최소 제곱 회귀 모델)의 경우, 입력 변수의 형식이 one hot encoding이므로 해당 형태로 변환한 후 모델을 훈련하고. 다른 모델은  일반적인 방식으로 훈련(model.fit) 및 예측(model.predict)을 수행합니다.

각 폴드에서 얻어진 평가 지표들의 평균을 계산하여 최종 결과를 사전 형태로 반환합니다.

In [ ]:
def binary_smote_cross_validation(model, x, y, seed, args):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

    metrics = ['precision', 'recall', 'f1', 'accuracy', 'auc']

    train_metrics = list(map(lambda x: 'train_' + x, metrics))
    val_metrics = list(map(lambda x: 'val_' + x, metrics))

    train_precision = []
    train_recall = []
    train_f1 = []
    train_accuracy = []
    train_auc = []

    val_precision = []
    val_recall = []
    val_f1 = []
    val_accuracy = []
    val_auc = []

    for train_idx, val_idx in skf.split(x, y):
        train_x, train_y = x.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = x.iloc[val_idx], y.iloc[val_idx]

        smote = SMOTE(random_state = args.smoteseed, k_neighbors = args.neighbor)
        train_x, train_y = smote.fit_resample(train_x, train_y)

        if type(model) == sklearn.cross_decomposition._pls.PLSRegression:
            model.fit(train_x, train_y)

            train_pred_score = model.predict(train_x)
            train_pred = np.where(train_pred_score < 0.5, 0, 1).reshape(-1)

            val_pred_score = model.predict(val_x)
            val_pred = np.where(val_pred_score < 0.5, 0, 1).reshape(-1)

        else:
            model.fit(train_x, train_y)

            train_pred = model.predict(train_x)
            train_pred_score = model.predict_proba(train_x)[:, 1]

            val_pred = model.predict(val_x)
            val_pred_score = model.predict_proba(val_x)[:, 1]

        train_precision.append(precision_score(train_y, train_pred))
        train_recall.append(recall_score(train_y, train_pred))
        train_f1.append(f1_score(train_y, train_pred))
        train_accuracy.append(accuracy_score(train_y, train_pred))
        train_auc.append(roc_auc_score(train_y, train_pred_score))

        val_precision.append(precision_score(val_y, val_pred))
        val_recall.append(recall_score(val_y, val_pred))
        val_f1.append(f1_score(val_y, val_pred))
        val_accuracy.append(accuracy_score(val_y, val_pred))
        val_auc.append(roc_auc_score(val_y, val_pred_score))

    result = dict(zip(train_metrics + val_metrics,
                      [np.mean(train_precision), np.mean(train_recall), np.mean(train_f1), np.mean(train_accuracy), np.mean(train_auc),
                       np.mean(val_precision), np.mean(val_recall), np.mean(val_f1), np.mean(val_accuracy), np.mean(val_auc)]))

    return(result)

**binary_smote_cross_validation** 함수는 이진 분류 문제에 대해 SMOTE 기법을 활용하여 교차 검증을 수행하는 함수입니다. **모델(model), 특성 데이터(x), 타겟 데이터(y), 시드 값(seed), 추가 인자(args)**를 입력으로 받습니다.

 앞서 multiclass_cross_valdation 함수에서 사용했던 **StratifiedKFold**를 사용하여 데이터를 5개의 폴드로 나눕니다. 이를 통해 각 클래스에 대한 비율을 유지한 체로 폴드를 생성됩니다.

 이후 각 폴드를 구성하는 데이터 셋에 대해 SMOTE 기법을 사용하여 훈련 데이터의 소수 클래스를 **오버샘플링합**니다. 이를 통해 불균형 데이터섯에 대한 처리를 진행할 수 있습니다.

이후, multiclass_cross_validation에서 사용한 방식과 동일하게, 각 폴드에서 모델을 훈련시키고, 훈련 및 검증 데이터에 대한 여러 평가 지표(정밀도, 재현율, F1 점수, 정확도, AUC)를 계산합니다. 또한, 각 폴드의 평가 지표들의 평균을 계산하여 최종 결과를 사전 형태로 반환합니다.

In [ ]:
def metric_mean(data, metric: str):
    mean_per_hp = list(map(lambda x: np.mean(x[1]), data[metric].items()))
    return mean_per_hp

**metric_mean** 함수는 주어진 데이터에서 특정 메트릭의 평균값을 계산하는 함수입니다.

data 매개변수는 평가 메트릭이 저장된 데이터 구조(예: 사전)를 받습니다.

metric 매개변수를 통해 계산하고자 하는 특정 메트릭의 이름을 문자열로 받아 data[metric].items()를 통해 해당 메트릭에 대한 모든 값들을 순회합니다.

각 값에 대해 numpy의 mean 함수를 사용하여 평균을 계산하고, 이를 리스트로 변환합니다.

결과적으로, 입력된 메트릭에 대한 평균값들의 리스트를 반환합니다.

In [ ]:
def print_best_param(val_result, metric: str):

    mean_list = metric_mean(val_result, metric)
    max_idx = mean_list.index(max(mean_list))

    best_param = val_result['model'][f'model{max_idx}']

    return best_param

**print_best_param**은 성능이 가장 좋은 하이퍼파라미터를 결정하는 데에 사용합니다.

모델의 검증 결과를 담고 있는 데이터를 val_result 파라미터로 받고, metric는 최적화하고자 하는 평가 지표의 이름을 문자열로 받습니다.

함수는 먼저 **metric_mean**함수를 호출하여 주어진 메트릭에 대한 각 매개변수 설정의 평균 점수를 계산합니다.

이후, 가장 높은 평균 점수를 가진 매개변수 설정의 인덱스(max_idx)를 찾고 val_result에서
해당 인덱스를 사용하여 최적의 매개변수를 검색하고 반환합니다.

In [ ]:
def load_val_result(path: str, tg_num: int, inhale_type: str, model: str, is_smote = True):
    if is_smote:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary_smote5/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
    else:
        try:
            with open(f'{path}/tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)
        except:
            with open(f'{path}tg{tg_num}_val_results/binary/{inhale_type}_{model}.json', 'r') as file:
                val_result = json.load(file)

    return val_result

**load_val_result** 함수는 JSON 형식의 검증 결과를 불러오는 함수입니다.

path, tg_num, inhale_type, model 매개변수들은 파일 경로와 파일 이름을 결정하는 데 사용되며 is_smote 매개변수는 SMOTE 기법을 사용한 결과를 불러올지, 사용하지 않은 결과를 불러올 지 결정해줍니다.

SMOTE 사용 여부(is_smote)에 따라 다른 파일 경로에서 JSON 파일을 불러옵니다.
try-except 블록을 통해 첫 번째 try에서 파일을 열지 못하면, except 블록에서 다른 경로의 파일을 시도합니다.

JSON 파일의 내용은 json.load 함수를 사용하여 파이썬 객체로 변환됩니다.